In [19]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim



In [20]:
# Load and preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None
    image = cv2.resize(image, (400, 200))  # Resize to standard size
    return image


In [21]:
from google.colab import drive
drive.mount('/content/drive') # Mount to the default /content/drive directory first

# Then you can access MyDrive
import os
image_folder_path = os.path.join('/content/drive/MyDrive', 'images-high-res')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!cp -r "/content/drive/My Drive/images-high-res" /content/

In [26]:

# Compare two images using SSIM
def compare_images(image1, image2):
    score, diff = ssim(image1, image2, full=True)
    diff = (diff * 255).astype("uint8")
    return score, diff

# Apply threshold and find contours
def detect_fake_pan(diff):
    _, thresh = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return len(contours) > 10  # If too many differences, likely fake

# Check if an image is tampered
def check_tampered_image(test_image_path, reference_image):
    test_image = preprocess_image(test_image_path)
    if test_image is None:
        return None, "Error loading image"

    score, diff = compare_images(reference_image, test_image)
    result = "Original" if score >= 0.9 else "Tampered"

    return score, result

# Process a folder containing PAN card images
def process_pan_folder(folder_path, reference_image_path):
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' not found!")
        return

    reference_image = preprocess_image(reference_image_path)
    if reference_image is None:
        print("Error: Could not load reference PAN card image.")
        return

    print(f"\nProcessing images in folder: {folder_path}\n")

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Process only image files
            score, result = check_tampered_image(file_path, reference_image)
            if score is not None:
                print(f"Image: {filename} | SSIM Score: {score:.2f} | Result: {result}")

# Path to the folder containing PAN card images
test_folder = r'/content/drive/MyDrive/images-high-res' # Change to your actual folder path
reference_image_path = "original.jpg"  # Reference PAN card

# Run the processing function
process_pan_folder(test_folder, reference_image_path)


Processing images in folder: /content/drive/MyDrive/images-high-res

Image: 16.png | SSIM Score: 0.25 | Result: Tampered
Image: 12.png | SSIM Score: 0.25 | Result: Tampered
Image: 11.png | SSIM Score: 0.24 | Result: Tampered
Image: 1.png | SSIM Score: 0.28 | Result: Tampered
Image: 10.png | SSIM Score: 0.29 | Result: Tampered
Image: 4.png | SSIM Score: 0.23 | Result: Tampered
Image: 31.png | SSIM Score: 0.25 | Result: Tampered
Image: 3.png | SSIM Score: 0.10 | Result: Tampered
Image: 29.png | SSIM Score: 0.12 | Result: Tampered
Image: 27.png | SSIM Score: 0.16 | Result: Tampered
Image: 28.png | SSIM Score: 0.20 | Result: Tampered
Image: 2.png | SSIM Score: 0.35 | Result: Tampered
Image: 23.png | SSIM Score: 0.13 | Result: Tampered
Image: 22.png | SSIM Score: 0.24 | Result: Tampered
Image: 24.png | SSIM Score: 0.28 | Result: Tampered
Image: 25.png | SSIM Score: 0.26 | Result: Tampered
Image: 26.png | SSIM Score: 0.33 | Result: Tampered
Image: 19.png | SSIM Score: 0.29 | Result: Tampere